In [65]:
import pickle

In [66]:
import os

In [67]:
os.getcwd()

'/lustre/work/statgrads/vpiyush2/MEnKF-ANN/Data_Generation/Doc2Vec_ANN'

In [68]:
with open('../Data//train_valid_test_splits_50.pkl', 'rb') as f:
    catch = pickle.load(f)

In [69]:
with open('doc2vec_ann_train_logits.pkl', 'rb') as f:
    catch_train_logits = pickle.load( f)

In [70]:
with open('doc2vec_ann_valid_logits.pkl', 'rb') as f:
    catch_valid_logits = pickle.load( f)

In [71]:
with open('doc2vec_ann_test_logits.pkl', 'rb') as f:
    catch_test_logits = pickle.load( f)

In [72]:
var_targets = 0.0001

In [73]:
reps = 50

In [74]:
from scipy.stats import norm

In [75]:
import tensorflow as tf

In [76]:
catch_train_logits_second = []
catch_train_probs_second = []
catch_train_labels_second = []
for i in range(0, reps):
    train_true_logits = catch_train_logits[i]
    train_logits_noise = norm(0,var_targets).rvs(train_true_logits.shape[0]).reshape(-1,1)
    train_logits_second = train_true_logits + train_logits_noise
    train_fudged_probs = tf.nn.sigmoid(train_logits_second).numpy()
    train_fudged_labels = (train_fudged_probs >= 0.5).astype(float)
    # train_fudged_labels = np.random.binomial(1, p=train_fudged_probs)
    catch_train_logits_second.append(train_logits_second)
    catch_train_probs_second.append(train_fudged_probs)
    catch_train_labels_second.append(train_fudged_labels)

In [77]:
catch_valid_logits_second = []
catch_valid_probs_second = []
catch_valid_labels_second = []
for i in range(0, reps):
    valid_true_logits = catch_valid_logits[i]
    valid_logits_noise = norm(0,var_targets).rvs(valid_true_logits.shape[0]).reshape(-1,1)
    valid_logits_second = valid_true_logits + valid_logits_noise
    valid_fudged_probs = tf.nn.sigmoid(valid_logits_second).numpy()
    valid_fudged_labels = (valid_fudged_probs >= 0.5).astype(float)
    catch_valid_logits_second.append(valid_logits_second)
    catch_valid_probs_second.append(valid_fudged_probs)
    catch_valid_labels_second.append(valid_fudged_labels)

In [78]:
catch_test_logits_second = []
catch_test_probs_second = []
catch_test_labels_second = []
for i in range(0, reps):
    test_true_logits = catch_test_logits[i]
    test_logits_noise = norm(0,var_targets).rvs(test_true_logits.shape[0]).reshape(-1,1)
    test_logits_second = test_true_logits + test_logits_noise
    test_fudged_probs = tf.nn.sigmoid(test_logits_second).numpy()
    test_fudged_labels = (test_fudged_probs >= 0.5).astype(float)
    # test_fudged_labels = np.random.binomial(1, p=test_fudged_probs)
    catch_test_logits_second.append(test_logits_second)
    catch_test_probs_second.append(test_fudged_probs)
    catch_test_labels_second.append(test_fudged_labels)

In [79]:
import numpy as np

In [80]:
for i in range(0, reps):
    print(np.mean(catch_train_labels_second[i] == catch[i][3].reshape(-1,1)))

1.0
0.9803921568627451
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9411764705882353
0.9803921568627451
1.0
1.0
1.0
1.0
1.0
0.9803921568627451
1.0
0.9803921568627451
1.0
1.0
1.0
1.0
0.9411764705882353
0.9803921568627451
1.0
1.0
0.9411764705882353
1.0
1.0
0.9607843137254902
1.0
1.0
1.0
1.0
0.9607843137254902
1.0
1.0
1.0
0.9803921568627451
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [81]:
rate = 0.5

In [82]:
lr = 1e-3

In [83]:
import gensim

In [84]:
doc2vec_dbow = gensim.models.doc2vec.Doc2Vec.load(r"..//doc2vec_dbow")

In [85]:
doc2vec_shape = doc2vec_dbow.dv.vectors.shape[1]

In [86]:
def doc2vec_ann(rate, training): 
    
    input_layer = tf.keras.layers.Input(shape = (doc2vec_shape))

    hidden_layer = tf.keras.layers.Dense(16, kernel_regularizer=tf.keras.regularizers.L1())
    
    hidden_output = hidden_layer(input_layer)
    
    drop_layer = tf.keras.layers.Dropout(rate)
    
    dropout_out = drop_layer(hidden_output, training = training)  

    pred_head = tf.keras.layers.Dense(1, kernel_regularizer=tf.keras.regularizers.L1())

    pred_output = pred_head(dropout_out)

    model = tf.keras.models.Model(input_layer, pred_output)
    
    model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                 optimizer = tf.keras.optimizers.Adam(learning_rate = lr), 
                 metrics=tf.keras.metrics.BinaryAccuracy())
    
    return model

In [87]:
samp_model = doc2vec_ann(0.75, False)

In [88]:
samp_model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 50)]              0         
                                                                 
 dense_4 (Dense)             (None, 16)                816       
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 833
Trainable params: 833
Non-trainable params: 0
_________________________________________________________________


In [89]:
# chk = attention_lstm_model(rate, False)

In [90]:
# chk.layers

In [91]:
def doc2vec_ann_training(idx):
    
    train_doc2vec = []
    for seq in catch[idx][0]:
        seq_txt = seq[0]
        seq_split = seq_txt.split(" ")
        train_doc2vec.append(doc2vec_dbow.infer_vector(seq_split))
    train_doc2vec = np.array(train_doc2vec)
    
    valid_doc2vec = []
    for seq in catch[idx][1]:
        seq_txt = seq[0]
        seq_split = seq_txt.split(" ")
        valid_doc2vec.append(doc2vec_dbow.infer_vector(seq_split))
    valid_doc2vec = np.array(valid_doc2vec)
    
    test_doc2vec = []
    for seq in catch[idx][2]:
        seq_txt = seq[0]
        seq_split = seq_txt.split(" ")
        test_doc2vec.append(doc2vec_dbow.infer_vector(seq_split))
    test_doc2vec = np.array(test_doc2vec)      

    doc2vec_ann_mod = doc2vec_ann(0.75, False)
    # model_word2vec = simple_lstm(False, model_cbow)
    # init_weights = first_lstm.get_weights()
    
    doc2vec_ann_mod.fit(train_doc2vec, catch_train_labels_second[idx], epochs = 2000, verbose = 0, 
                  callbacks = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 30,
                                                              restore_best_weights=True), 
                      validation_data = (valid_doc2vec, catch_valid_labels_second[idx]))
    
    X_train_logits = doc2vec_ann_mod.predict(train_doc2vec, verbose = 0)
    X_valid_logits = doc2vec_ann_mod.predict(valid_doc2vec, verbose = 0)
    X_test_logits = doc2vec_ann_mod.predict(test_doc2vec, verbose = 0)
    
    doc2vec_ann_mod.save("Real_World_Doc2Vec_ANN_Second" + "//" + "Model_" + str(idx))
    
    doc2vec_ann_mod_extract = tf.keras.models.Model(doc2vec_ann_mod.input, doc2vec_ann_mod.layers[1].output)
    
    X_train_emb = doc2vec_ann_mod_extract.predict(train_doc2vec, verbose = 0)
    X_valid_emb = doc2vec_ann_mod_extract.predict(valid_doc2vec, verbose = 0)
    X_test_emb = doc2vec_ann_mod_extract.predict(test_doc2vec, verbose = 0)
    
    
    print(doc2vec_ann_mod.evaluate(test_doc2vec, catch[idx][5]), flush = True)
    
    return X_train_emb, X_valid_emb, X_test_emb, X_train_logits, X_valid_logits , X_test_logits

In [92]:
from joblib import Parallel, delayed

In [93]:
second_lstm = Parallel(n_jobs=15, verbose = 10, backend = "loky")(delayed(doc2vec_ann_training)( i) for i in range(reps))

[Parallel(n_jobs=15)]: Using backend LokyBackend with 15 concurrent workers.
2023-08-14 20:52:23.472707: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-14 20:52:23.472707: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-14 20:52:23.472706: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-14 20:52:23.472716: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-14 20:52:23.472707: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-14 20:52:23.472795: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-14 20:52:23.472795: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-14 20:52:23.472816: W tensorflow/compiler/tf2te

1/1 [==============================] - 0s 472ms/step - loss: 0.6339 - binary_accuracy: 0.7500
[0.6339040994644165, 0.75]


1/1 [==============================] - 0s 400ms/step - loss: 0.4378 - binary_accuracy: 0.8750
[0.4377652704715729, 0.875]


[Parallel(n_jobs=15)]: Done   2 tasks      | elapsed:   28.9s


1/1 [==============================] - 0s 413ms/step - loss: 0.4530 - binary_accuracy: 1.0000
[0.4530335068702698, 1.0]


...

1/1 [==============================] - 0s 229ms/step - loss: 0.7967 - binary_accuracy: 0.6250
[0.796675980091095, 0.625]


1/1 [==============================] - 0s 213ms/step - loss: 0.6202 - binary_accuracy: 0.8750
[0.6201861500740051, 0.875]


1/1 [==============================] - 0s 210ms/step - loss: 0.4127 - binary_accuracy: 0.8750
[0.4126754403114319, 0.875]


1/1 [==============================] - 0s 208ms/step - loss: 0.4378 - binary_accuracy: 0.8750
[0.4378020763397217, 0.875]


[Parallel(n_jobs=15)]: Done  50 out of  50 | elapsed:  2.8min finished


In [94]:
with open('doc2vec_ann_second.pkl', 'wb') as f:
    pickle.dump(second_lstm, f)

In [97]:
second_lstm[0][2].shape

(8, 16)